In [10]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# dir_path = './' #local
dir_path = './drive/MyDrive/BAKA/' #colab

In [11]:
#uncomment in colab
from google.colab import drive
drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '10' + '/' + '0.txt', header=None, sep=' ')
df.drop(df.index[100:])
filler = np.zeros((100-len(df), 31))
df = df.append(pd.DataFrame(filler), ignore_index=True )
df
# df = df.drop(columns=[31])
# sp_df = np.array_split(df, 2)
# sp_df[0]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,164.280661,176.325903,178.121795,173.909634,179.942562,165.068138,179.880849,169.886642,179.693632,178.665290,0.428724,-0.367061,0.589080,0.470112,-0.276933,0.597770,0.503002,-0.261926,0.597986,0.521324,-0.265650,0.589049,0.564965,-0.296812,0.587671,0.523166,-0.401421,0.608364,13.702051,7.694128,16.043218
1,165.122590,175.624965,179.600487,179.520481,177.638911,166.203109,168.936561,175.217537,179.756639,179.472871,0.429849,-0.363784,0.589629,0.474212,-0.278814,0.602315,0.502064,-0.260462,0.597467,0.521769,-0.265654,0.589785,0.563702,-0.294720,0.586825,0.522370,-0.401031,0.608108,14.720415,4.666922,14.926564
2,165.078781,176.016382,179.983630,179.527066,178.258843,166.794220,169.728408,179.934052,179.742772,179.466085,0.430263,-0.362915,0.589836,0.473503,-0.277637,0.601022,0.501912,-0.259693,0.597525,0.520842,-0.264959,0.588971,0.563148,-0.294356,0.586612,0.522276,-0.401130,0.608519,14.914579,3.442307,12.785423
3,160.916565,172.198336,179.873084,178.794688,171.190540,169.025568,169.618909,178.304716,179.278919,176.811213,0.436674,-0.352632,0.587307,0.476525,-0.275765,0.603303,0.501905,-0.259208,0.597259,0.520186,-0.263787,0.588143,0.564566,-0.294643,0.588136,0.521309,-0.400198,0.607621,20.174759,4.338054,7.668797
4,159.852724,179.822872,179.118311,179.866069,178.201757,168.584125,175.381159,179.493500,178.231357,178.974019,0.437839,-0.352005,0.587716,0.479530,-0.276220,0.605988,0.502092,-0.258766,0.597192,0.518232,-0.261848,0.585181,0.557927,-0.292103,0.584925,0.520796,-0.399540,0.607557,12.097098,5.726537,6.220917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
96,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
97,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [12]:
files = os.listdir(dir_path + 'DataCollection')
files

['6', '9', '7', '10', '8', '1', '0', '5', '4', '3', '2']

In [9]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[100:])
        if len(df) < 100:
            filler = np.zeros((100-len(df), 31))
            df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_test.append(df.to_numpy())
        y_test.append(int(i));
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        df = df.drop(df.index[100:])
        if len(df) < 100:
            filler = np.zeros((100-len(df), 31))
            df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests)
    
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)

200   40
200   40
200   40
200   40
200   40
602   120
302   60
422   84
410   82
302   60
512   102


In [13]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)

print(x_train)

x_train.shape:  (2842, 100, 31)
y_train.shiape:  (2842,)
x_test.shape:  (708, 100, 31)
y_test.shape:  (708,)
[[[171.05097271 179.077056   108.14802011 ...  19.93247993   8.68486513
   143.25822471]
  [171.03335405 178.99149018 107.83135339 ...  19.72261924   8.60300522
   143.08174588]
  [170.65335795 159.33471712 102.94041811 ...  19.80701946   7.45194274
   146.13562988]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[169.0399894  178.81230926 143.30123449 ...   8.36777393  16.31071094
   141.50138724]
  [168.67810689 178.69997299 141.9033215  ...   7.42614716  18.02363086
   140.09869202]
  [168.44696217 178.73440153 140.70784505 ...   6.36973187  18.08955845
   139.45877392]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0

In [14]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [15]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [16]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (2842, 100, 31)
y_train.shiape:  (2842,)
x_test.shape:  (708, 100, 31)
y_test.shape:  (708,)


In [20]:
model = Sequential()
model.add(LSTM(units=100, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(11, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 100, 100)          52800     
_________________________________________________________________
batch_normalization_8 (Batch (None, 100, 100)          400       
_________________________________________________________________
dropout_8 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
batch_normalization_9 (Batch (None, 100, 100)          400       
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 100, 100)         

In [21]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=200,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=80,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/200
32/32 [==============================] - 13s 240ms/step - loss: 2.4255 - accuracy: 0.2728 - val_loss: 2.2895 - val_accuracy: 0.4421
Epoch 2/200
32/32 [==============================] - 7s 205ms/step - loss: 0.9333 - accuracy: 0.6618 - val_loss: 2.1823 - val_accuracy: 0.4877
Epoch 3/200
32/32 [==============================] - 7s 205ms/step - loss: 0.7489 - accuracy: 0.7171 - val_loss: 2.0377 - val_accuracy: 0.5509
Epoch 4/200
32/32 [==============================] - 7s 204ms/step - loss: 0.6943 - accuracy: 0.7297 - val_loss: 1.8293 - val_accuracy: 0.6386
Epoch 5/200
32/32 [==============================] - 7s 204ms/step - loss: 0.6548 - accuracy: 0.7439 - val_loss: 1.5479 - val_accuracy: 0.6526
Epoch 6/200
32/32 [==============================] - 7s 204ms/step - loss: 0.6119 - accuracy: 0.7524 - val_loss: 1.2813 - val_accuracy: 0.6281
Epoch 7/200
32/32 [==============================] - 7s 204ms/step - loss: 0.5839 - accuracy: 0.7627 - val_loss: 0.9907 - val_accuracy: 0.680

In [22]:
# model.save(dir_path + 'Models/gestures.h5')

In [23]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=24)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_dynamic', save_format='tf')

Evaluate on test data
9/9 [==============================] - 1s 77ms/step - loss: 0.9536 - accuracy: 0.8121
test loss, test acc: [0.9535993933677673, 0.8121469020843506]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_dynamic/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_dynamic/assets


In [24]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [26]:
min_max_scaler.data_max_

array([180.        , 179.99999233, 180.        , 179.99998317,
       179.99994529, 179.999     , 179.99997165, 180.        ,
       179.99994445, 179.99998066, 404.416     , 719.193     ,
       570.106     , 433.43      , 778.684     , 578.49      ,
       423.871     , 791.146     , 551.114     , 433.991     ,
       783.643     , 520.983     , 443.729     , 762.745     ,
       508.462     , 383.431     , 723.459     , 526.573     ,
       179.78636976, 179.25331623, 179.54454478])

In [30]:
model.input

<KerasTensor: shape=(None, 100, 31) dtype=float64 (created by layer 'lstm_8_input')>